<a href="https://colab.research.google.com/github/jmhuer/optimization_tools/blob/main/homework5/Homework_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 5
In this problem set you will implement the Conjugate Gradient algorithm, also known as the Frank Wolfe algorithm, and you will compare it to algiorithms you have implemented on previous homeworks. You will also play with Mirror Descent.

In [5]:
# ! git clone https://github.com/jmhuer/optimization_tools
import optimization_tools.utils 


import re
import os
import subprocess

a = 'https://drive.google.com/file/d/1YDmMRdgRRJSKaWGR7c0g6mzj73Bzf9MU/view?usp=sharing'
b ='https://drive.google.com/file/d/1ZdnrCU7YotTAU0duRHTvUUF-k23lG7Wd/view?usp=sharing'
lg_news = 'https://drive.google.com/file/d/1mjADAoaHTISJYFuUzhWU_LkFFtdrw3ZF/view?usp=sharing'

def download_gdrive(share_link, local_name, print_stout=False):
  id = re.search('d/(.*?)/view', share_link).group(1)
  command = "wget --load-cookies /tmp/cookies.txt \"https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \'https://docs.google.com/uc?export=download&id={}\' -O- | sed -rn \'s/.*confirm=([0-9A-Za-z_]+).*/\\1\\n/p')&id={}\"  -O \'{}\' && rm -rf /tmp/cookies.txt".format(id,id,local_name)
  returned_value = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT) 
  if print_stout: print(returned_value.stdout.decode("utf-8"))
  else: print("Download Complete: " + local_name) 

download_gdrive(a, "a.npy")
download_gdrive(b, "b.npy")
download_gdrive(lg_news, "logistic_news.zip")

Cloning into 'optimization_tools'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 20 (delta 1), reused 20 (delta 1), pack-reused 0
Unpacking objects: 100% (20/20), done.
Download Complete: a.npy
Download Complete: b.npy
Download Complete: logistic_news.zip


In [4]:
# %rm -rf /content/optimization_tools

# Problem 1: LASSO via Frank Wolfe
Recall the least squares problem with $\ell^1$ regularization from the previous two homeworks:
$$
\min_x \left[f(x) = \frac{1}{2}\|{Ax-b}\|_2^2 + \lambda \|{x}\|_1 \right]
$$

So far you have used the subgradient method, ISTA and FISTA (proxima gradient, and accelerated proximal gradient) to solve this problem. Now you will use Frank-Wolfe.

As we cover in the lectures, the Frank-Wolfe (or conditional gradient) algorithm minimizes a smooth function $f(x)$ subject to a convex constraint $x \in \mathcal{X}$.

When it is easy to minimize a linear function over $\mathcal{X}$, Frank-Wolfe (FW) has several advantages including that it often produces sparse iterates and does not require a projection step to stay within $\mathcal{X}$. In order to apply FW to the LASSO problem above, we can reformulate the problem as 

\begin{eqnarray*}
\min_{{x}}: && \frac{1}{2}\|A{x}-{b}\|_2^2, \\
\mbox{s.t. } && \|{x}\|_1 \leq \gamma.
\end{eqnarray*}

This formulation is equivalent to the original LASSO formulation for a suitable value of $\gamma$ (and depending on the $\lambda$ in the original LASSO formulation). 

Run the FW algorithm for $10^4$ steps and find a $\gamma$ that performs well empirically (you might consider the $\ell^1$ norm of your solution from previous homework. Compare the results to sub-gradient method implemented in the previous homework. Again, separately record the (unsquared) error $\|A{x}_t-{b}\|$ and the regularization term $\|{x}\|_1$.

In [ ]:
import numpy as np
import numpy.random as rn
import numpy.linalg as la
import matplotlib.pyplot as plt
import time

A = np.load("A.npy")
b = np.load("b.npy")

#Problem 2: Robust Regression, PSGD and Mirror Descent
Consider the problem of robust regression, where some small number of measurements have been potentially completely corrupted. One way to formulate an optimization problem to solve this robust regression is as follows:

\begin{eqnarray*}
\min_{\beta}: && \|X \beta - \mathbf{y}\|_1 \\
{\rm s.t.}: && \beta \in \mathcal{X}.
\end{eqnarray*}

The rationale for this formulation stems from the idea that because of the $\ell^1$-error, huge errors are not disproportionally penalized, as they would be in the squared error formulation (this is the formulation we have worked with before, including in the previous problem), and therefore the optimal solution is less sensitive to outliers. You will solve this problem using Projected Subgradient Descent, and also Mirror Descent. Let the constraint set be the simplex: 
$$
\mathfrak{X} = \{\beta \,:\, \beta \geq 0, \, \sum \beta_i = 1\}.
$$





##Part (A)
Write down the update for projected gradient descent. 

##Part (B) 
Write down the mirror descent update. For this, we will use the mirror map $\Phi(\beta) = \sum \beta_i \log \beta_i$. Compute the Bregman divergence, $D_{\phi}(\mathbf{\beta},\mathbf{z})$ explicitly. <br>
(We use $\mathbf{z}$ here to not confuse this variable with $\mathbf{y}$ in the robust regression problem above. $\mathbf{z}$ is the intermediate term used in Mirror Descent, which was commonly denoted as $\mathbf{y}$ in the lectures.)

##Part (C)
Using the data in X.npy and y.npy, and using stepsizes of your choosing, compare the projected subgradient method with mirror descent. What is $\beta$? Plot the objective above against iterations for both methods (in a single plot). 

In [ ]:
X = np.load("X.npy")
y = np.load("y.npy")